In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import librosa.display
import numpy as np
import librosa
import tensorflow as tf
import glob

### If you have been save the data, you don't have to preprocessing and save the data

In [2]:
drone_path = '../../data/drone/*.wav'
background_path = '../../data/background/*.wav'

drone_files = glob.glob(drone_path)
background_files = glob.glob(background_path)

In [3]:
CHUNK_SIZE = 8192
SR = 44100
N_MFCC = 16

In [4]:
def load(files, sr=44100):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw

In [5]:
drone_raw = load(drone_files)
background_raw = load(background_files)

(22813237,)
(20257033,)


# Data Processing

In [6]:
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=44100, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc, n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0], mfcc_slice.shape[1]))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [7]:
mfcc_drone, y_drone = mfcc4(drone_raw, 1)
mfcc_background, y_background = mfcc4(background_raw, 0)

print(mfcc_drone.shape, y_drone.shape)
print(mfcc_background.shape, y_background.shape)

(22813237,)
small end: (16, 14)
small end: (16, 6)
(20257033,)
small end: (16, 13)
small end: (16, 5)
(5568, 16, 16) (5568,)
(4944, 16, 16) (4944,)


In [8]:
X = np.concatenate((mfcc_drone, mfcc_background), axis=0)
#X = np.concatenate((mfcc_drone), axis=0)
#X = X.reshape(-1, 16,16,1)
y = np.hstack((y_drone, y_background))
#y = np.hstack(y_drone)
print(X.shape, y.shape)

(10512, 16, 16) (10512,)


In [9]:
n_labels = y.shape[0]
n_unique_labels = 2
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)

(10512, 2)


In [10]:
# Split data
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_encoded, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [11]:
print(X_train.shape, X_test.shape)
print(X_val.shape, y_val.shape)
print(y_train.shape, y_test.shape)

(6727, 16, 16) (2103, 16, 16)
(1682, 16, 16) (1682, 2)
(6727, 2) (2103, 2)


In [12]:
# Save Data
np.save('../../data/X_train', X_train)
np.save('../../data/X_test', X_test)
np.save('../../model/X_val', X_val)
np.save('../../model/y_val', y_val)
np.save('../../data/y_train', y_train)
np.save('../../data/y_test', y_test)

### Until this part

In [13]:
# Load Data
X_train = np.load('../../data/X_train.npy')
X_test = np.load('../../data/X_test.npy')
X_val = np.load('../../model/X_val.npy')
y_val = np.load('../../model/y_val.npy')
y_train = np.load('../../data/y_train.npy')
y_test = np.load('../../data/y_test.npy')

# Experiment 3 - One convolutional layer /w no dropout

##Experiment 3-2
- learning rate 0.005
- pooling stride 1x1
- #filter 1
- best result among every other settings
- cost kept fluctuated during training. (0.8 -> 1.3) -- why is that?

In [14]:
tf.reset_default_graph()

In [15]:
n_mfcc = 16
n_frame = 16
n_classes = 2
n_channels = 1

learning_rate = 0.0002  # 0.005
training_epochs = 500 # 수정해봐

# Layer

In [16]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
X = tf.reshape(X, [-1, n_mfcc, n_frame, n_channels])
Y = tf.placeholder(tf.float32, shape=[None,n_classes])

conv1 = tf.layers.conv2d(inputs=X, filters=1, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                padding="SAME", strides=1)
# dropout넣어야하나
conv2 = tf.layers.conv2d(inputs=pool1, filters=1, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                padding="SAME", strides=1)
# 여기도
flat = tf.reshape(pool2, [-1, 16*16*1])
dense2 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
logits = tf.layers.dense(inputs=dense2, units=2)

In [17]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [18]:
Y_pred = tf.contrib.layers.fully_connected(logits,n_classes,activation_fn = None)

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [20]:
X_train2 = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test2 = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
X_val2 = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)

In [21]:
# model save
model_path = '../../model/CNN/cnn_model'
saver = tf.train.Saver()

# Trainning

In [22]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import itertools as it

In [23]:
###########################
batch_size = 32
cost_history = np.empty(shape=[1], dtype=float)

for epoch in range(training_epochs):#training epoch 500 / batch_size 128 --> acc 90%
    avg_cost = 0
    val_avg_cost =0
    total_batch = int(y_train.shape[0] / batch_size)
    for i in range(0, y_train.shape[0], batch_size):
        feed_dict={X:X_train2[i:i+batch_size,:,:,:], Y:y_train[i:i+batch_size,:]}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        cost_history = np.append(cost_history,cost)
        avg_cost += c/total_batch 
    
    y_pred = sess.run(logits, feed_dict={X:X_val2})
    y_pred = sess.run(tf.argmax(y_pred,1))
    y_true = y_val
        
    y_true = sess.run(tf.argmax(y_true,1))
    print(len(y_pred),end=' ')
    print('Epoch:', '%04d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost), 'val = ','%f' %(accuracy_score(y_true, y_pred)) )
saver.save(sess, model_path)

1682 Epoch: 0001 cost =  2.776207676 val =  0.960761
1682 Epoch: 0002 cost =  0.205617780 val =  0.964328
1682 Epoch: 0003 cost =  0.162430421 val =  0.976813
1682 Epoch: 0004 cost =  0.185757574 val =  0.980380
1682 Epoch: 0005 cost =  0.178073363 val =  0.958977
1682 Epoch: 0006 cost =  0.201009900 val =  0.969679
1682 Epoch: 0007 cost =  0.166507518 val =  0.978002
1682 Epoch: 0008 cost =  0.193725958 val =  0.987515
1682 Epoch: 0009 cost =  0.137854683 val =  0.986920
1682 Epoch: 0010 cost =  0.142151556 val =  0.970273
1682 Epoch: 0011 cost =  0.188005917 val =  0.986326
1682 Epoch: 0012 cost =  0.112348739 val =  0.986920
1682 Epoch: 0013 cost =  0.090218548 val =  0.988109
1682 Epoch: 0014 cost =  0.076901870 val =  0.991677
1682 Epoch: 0015 cost =  0.063762991 val =  0.985731
1682 Epoch: 0016 cost =  0.059946184 val =  0.986920
1682 Epoch: 0017 cost =  0.054682031 val =  0.987515
1682 Epoch: 0018 cost =  0.054143201 val =  0.984542
1682 Epoch: 0019 cost =  0.053038884 val =  0.

1682 Epoch: 0156 cost =  0.000000279 val =  0.994649
1682 Epoch: 0157 cost =  0.000000250 val =  0.994649
1682 Epoch: 0158 cost =  0.000000221 val =  0.994649
1682 Epoch: 0159 cost =  0.000000195 val =  0.994649
1682 Epoch: 0160 cost =  0.000000175 val =  0.994649
1682 Epoch: 0161 cost =  0.000000148 val =  0.994649
1682 Epoch: 0162 cost =  0.000000133 val =  0.994649
1682 Epoch: 0163 cost =  0.000000117 val =  0.994649
1682 Epoch: 0164 cost =  0.000000105 val =  0.994649
1682 Epoch: 0165 cost =  0.000000094 val =  0.994649
1682 Epoch: 0166 cost =  0.000000084 val =  0.994649
1682 Epoch: 0167 cost =  0.000000075 val =  0.994649
1682 Epoch: 0168 cost =  0.000000068 val =  0.994649
1682 Epoch: 0169 cost =  0.000000062 val =  0.994649
1682 Epoch: 0170 cost =  0.000000054 val =  0.995244
1682 Epoch: 0171 cost =  0.000000050 val =  0.995244
1682 Epoch: 0172 cost =  0.000000044 val =  0.995244
1682 Epoch: 0173 cost =  0.000000041 val =  0.995244
1682 Epoch: 0174 cost =  0.000000036 val =  0.

1682 Epoch: 0311 cost =  0.000000004 val =  0.993460
1682 Epoch: 0312 cost =  0.000000004 val =  0.993460
1682 Epoch: 0313 cost =  0.000000004 val =  0.993460
1682 Epoch: 0314 cost =  0.000000003 val =  0.993460
1682 Epoch: 0315 cost =  0.000000003 val =  0.992866
1682 Epoch: 0316 cost =  0.000000003 val =  0.993460
1682 Epoch: 0317 cost =  0.000000002 val =  0.993460
1682 Epoch: 0318 cost =  0.000000002 val =  0.993460
1682 Epoch: 0319 cost =  0.000000002 val =  0.993460
1682 Epoch: 0320 cost =  0.000000002 val =  0.994055
1682 Epoch: 0321 cost =  0.000000002 val =  0.993460
1682 Epoch: 0322 cost =  0.000000001 val =  0.994055
1682 Epoch: 0323 cost =  0.000000001 val =  0.994055
1682 Epoch: 0324 cost =  0.000000001 val =  0.994055
1682 Epoch: 0325 cost =  0.000000001 val =  0.994649
1682 Epoch: 0326 cost =  0.000000001 val =  0.994649
1682 Epoch: 0327 cost =  0.074835549 val =  0.992866
1682 Epoch: 0328 cost =  0.046700145 val =  0.994649
1682 Epoch: 0329 cost =  0.015635779 val =  0.

1682 Epoch: 0466 cost =  0.000004706 val =  0.994055
1682 Epoch: 0467 cost =  0.000003701 val =  0.994055
1682 Epoch: 0468 cost =  0.000003405 val =  0.994055
1682 Epoch: 0469 cost =  0.000003096 val =  0.994055
1682 Epoch: 0470 cost =  0.000002759 val =  0.994055
1682 Epoch: 0471 cost =  0.000002902 val =  0.994055
1682 Epoch: 0472 cost =  0.000002351 val =  0.994055
1682 Epoch: 0473 cost =  0.000002211 val =  0.994055
1682 Epoch: 0474 cost =  0.000002032 val =  0.994055
1682 Epoch: 0475 cost =  0.000001863 val =  0.994055
1682 Epoch: 0476 cost =  0.000001788 val =  0.994055
1682 Epoch: 0477 cost =  0.000001532 val =  0.994055
1682 Epoch: 0478 cost =  0.000001341 val =  0.994055
1682 Epoch: 0479 cost =  0.000001229 val =  0.994055
1682 Epoch: 0480 cost =  0.000001138 val =  0.994055
1682 Epoch: 0481 cost =  0.000001068 val =  0.994055
1682 Epoch: 0482 cost =  0.000000985 val =  0.994055
1682 Epoch: 0483 cost =  0.000000945 val =  0.994055
1682 Epoch: 0484 cost =  0.000000858 val =  0.

'../../model/CNN/cnn_model'

## Prediction

In [24]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test2})
y_true = sess.run(tf.argmax(y_test,1))

In [25]:
# Print Result

from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.991
Accuracy:  0.9914407988587732
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       966
           1       0.99      1.00      0.99      1137

   micro avg       0.99      0.99      0.99      2103
   macro avg       0.99      0.99      0.99      2103
weighted avg       0.99      0.99      0.99      2103

[[ 953   13]
 [   5 1132]]
